In [123]:
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()
collisions = pd.read_csv("../data/preprocessed-colisions.csv")
collisions.head(3).transpose()

,0,1,2
CRASH_DATE,09/06/2020,09/20/2020,09/24/2020
CRASH_TIME,18:05,9:14,22:00
BOROUGH,NaN,NaN,MANHATTAN
ZIP_CODE,NaN,NaN,10039.0
LATITUDE,40.771038,40.722095,40.824757
LONGITUDE,-73.83413,-73.77772,-73.94052
TOTAL_INJURED,0.0,0.0,0.0
TOTAL_KILLED,1.0,0.0,0.0
PEDESTRIANS_INJURED,0,0,0
PEDESTRIANS_KILLED,0,0,0


In [124]:
weather_original = pd.read_csv("../data/weather.csv")
weather = weather_original[['datetime', 'temp', 'precip', 'windspeed', 'humidity', 'cloudcover', 'conditions']]
weather['datetime'] = pd.to_datetime(weather['datetime'])
weather.head()

/var/folders/_6/8p4ncq_d54d7m8sx_m9m5y0r0000gn/T/ipykernel_19432/4123577636.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather['datetime'] = pd.to_datetime(weather['datetime'])


,datetime,temp,precip,windspeed,humidity,cloudcover,conditions
0,2018-06-01,21.6,0.28200,12.6,86.8,65.9,"Rain, Partially cloudy"
1,2018-06-02,25.1,0.34600,22.3,74.0,35.4,"Rain, Partially cloudy"
2,2018-06-03,17.0,2.92900,24.1,75.0,92.7,"Rain, Overcast"
3,2018-06-04,16.8,3.91978,16.7,76.6,71.6,"Rain, Partially cloudy"
4,2018-06-05,19.8,0.00000,25.9,60.7,35.7,Partially cloudy


In [125]:
coll_weather = pd.DataFrame({'datetime': collisions["CRASH_DATE"]})
coll_weather['datetime'] = pd.to_datetime(coll_weather['datetime'])
coll_weather = coll_weather.groupby(['datetime']).size().reset_index(name='collisions')
coll_weather = pd.merge(coll_weather, weather, on='datetime')
coll_weather['year'] = coll_weather['datetime'].dt.year
coll_weather.head()

,datetime,collisions,temp,precip,windspeed,humidity,cloudcover,conditions,year
0,2018-06-01,751,21.6,0.28200,12.6,86.8,65.9,"Rain, Partially cloudy",2018
1,2018-06-02,622,25.1,0.34600,22.3,74.0,35.4,"Rain, Partially cloudy",2018
2,2018-06-03,525,17.0,2.92900,24.1,75.0,92.7,"Rain, Overcast",2018
3,2018-06-04,698,16.8,3.91978,16.7,76.6,71.6,"Rain, Partially cloudy",2018
4,2018-06-05,688,19.8,0.00000,25.9,60.7,35.7,Partially cloudy,2018


In [126]:
alt.Chart(coll_weather, width=500).transform_window(
    index='count()'
).transform_fold(
    ['temp', 'precip', 'windspeed', 'humidity', 'cloudcover',]
).mark_line().encode(
    x='key:N',
    y='value:Q',
    color='year:N',
    detail='index:N',
    opacity=alt.value(0.5)
)

alt.Chart(...)

In [127]:
alt.Chart(coll_weather).transform_window(
    index='count()'
).transform_fold(
    ['temp', 'windspeed', 'collisions', 'humidity', 'cloudcover']
).transform_joinaggregate(
     min='min(value)',
     max='max(value)',
     groupby=['key']
).transform_calculate(
    minmax_value=(datum.value-datum.min)/(datum.max-datum.min),
    mid=(datum.min+datum.max)/2
).mark_line().encode(
    x='key:N',
    y='minmax_value:Q',
    color='year:N',
    detail='index:N',
    opacity=alt.value(0.5)
).properties(width=500)

alt.Chart(...)

Heatmap

In [128]:
alt.Chart(coll_weather).mark_rect().encode(
    x=alt.X('temp:O', bin=True),  # Binarize temperature
    y=alt.Y('windspeed:O', bin=True),  # Binarize windspeed
    color='average(collisions):Q',
)

alt.Chart(...)

In [132]:
alt.Chart(coll_weather).mark_point().encode(
    alt.X('temp:Q').title('Average Daily Temperature (C)').scale(domain=[15, 31]),
    alt.Y('windspeed:Q').title('Avearge Daily Windspeed (km/h)'),
    alt.Color('conditions'),#.scale(scale),
    alt.Size('collisions').scale(domain=[196, 845]),
).properties(
    width=600,
    height=400
)

alt.Chart(...)

In [133]:
#descirve the windspeed
coll_weather['windspeed'].describe()

count    244.000000
mean      19.365574
std        6.123735
min        8.800000
25%       14.675000
50%       18.400000
75%       23.525000
max       43.900000
Name: windspeed, dtype: float64

In [ ]:
alt.Chart(coll_weather).mark_point().encode(
    alt.X('temp:Q', title='Average Daily Temperature (C)'),
    alt.Y('windspeed:Q', title='Average Daily Windspeed (km/h)'),
    alt.Color('conditions'),
    alt.Size('collisions:Q', 
             scale=alt.Scale(domain=alt.Calculate("extent(datum['collisions'])")))
).properties(
    width=600,
    height=400
)

AttributeError: module 'altair' has no attribute 'Calculate'